In [2]:
import numpy as np
from itertools import combinations
from sage.all import QQ, NumberField, PolynomialRing

In [3]:
def find_feasible_vertices(A, b):
    """Find vertices of polytope defined by A @ x ≤ b"""
    num_vars = A.shape[1]
    vertices = []
    
    for indices in combinations(range(len(A)), num_vars):
        A_sub = A[list(indices)]
        b_sub = b[list(indices)]
        
        try:
            vertex = np.linalg.solve(A_sub, b_sub)
            if np.all(A @ vertex <= b + 1e-6):
                vertices.append(vertex)
        except np.linalg.LinAlgError:
            continue
    
    return np.unique(np.round(vertices, decimals=6), axis=0) if vertices else np.array([])


def twoParts(alpha,embeddings):
    """FIND ALL TWO PARTITION OF ALPHA"""


    """FEILD INPUT Calculation"""
  
    K = alpha.parent().fraction_field()  

    if not K.is_totally_real():
        raise ValueError("Field must be totally real")
    
    O = K.maximal_order()
    basis = O.basis()
    """FEILD INPUT Calculation Over"""

    """BOUNDS CALCULATION"""
    embedding_data = []
    for sig in embeddings:
        a_img = sig(basis[0]) if len(basis) > 0 else 0
        b_img = sig(basis[1]) if len(basis) > 1 else 0
        c_img = sig(basis[2]) if len(basis) > 2 else 0
        alpha_img = sig(alpha)
        embedding_data.append((a_img,b_img,c_img, alpha_img))
 
    A_ineq = []
    b_ineq = []
    epsilon = 10^(-6)
    
    for (a,b,c,Alpha) in embedding_data:
        A_ineq.append([float(a),float(b),float(c)])
        b_ineq.append(float(Alpha) - epsilon)
        A_ineq.append([-float(a),-float(b), -float(c)])
        b_ineq.append(-epsilon)

    A_ineq = np.array(A_ineq)
    b_ineq = np.array(b_ineq)
    
    vertices = find_feasible_vertices(A_ineq, b_ineq)
    
    if not vertices.size:
        return 0, []
    
    bound=[]
    
    for i in range(3):
        a=[]
        a.append(np.floor(np.min(vertices[:,i])))
        a.append(np.ceil(np.max(vertices[:,i])))
        bound.append(a)
  
    k1_min, k1_max = bound[0][0],bound[0][1]
    k2_min, k2_max = bound[1][0],bound[1][1]
    k3_min, k3_max = bound[2][0],bound[2][1]

    """BOUNDS CALCULATION OVER"""

    """FINDING PARTITIONS"""
    partitions = []
    for k1 in range(int(k1_min),int(k1_max)+1):
        for k2 in range(int(k2_min),int(k2_max)+1):
            for k3 in range(int(k3_min),int(k3_max)+1):
                beta= k1*basis[0] + k2*basis[1] + k3*basis[2]
         
                Y= alpha - beta
            
                if beta == 0 or Y == 0:
                    continue
            
                if beta.is_totally_positive() and Y.is_totally_positive():
                    partitions.append((beta, Y))
    
    unique_partitions = []
    seen = set()
    for pair in partitions:
        if pair not in seen and pair[::-1] not in seen:
            seen.add(pair)
            unique_partitions.append(pair)
    
    return unique_partitions


def allParts(alpha, embeddings, memo=None):
    """
    Recursively find all partitions of alpha into sums of totally positive elements.
    Stops recursion when no further partitions are possible.
    Returns the number of unique partitions.
    Memo store all partitions of parts of alpha and alpha
    """
    if memo is None:
        memo = {}                               
    
    alpha_key = tuple(alpha.list())
    if alpha_key in memo:
        return memo[alpha_key]
    
    if not alpha.is_totally_positive():
        raise ValueError("alpha is not totally positive")
    
    partitions = twoParts(alpha, embeddings)

    all_partitions = []
    
    for beta, gamma in partitions:
        all_partitions.append([beta, gamma])
    
        beta_partitions = allParts(beta, embeddings, memo)
        for p in beta_partitions:
            all_partitions.append(p + [gamma])
        
        gamma_partitions = allParts(gamma, embeddings, memo)
        for p in gamma_partitions:
            all_partitions.append([beta] + p)

    unique_partitions = []
    seen = set()
    for partition in all_partitions:
        sorted_part = tuple(sorted(tuple(x.list()) for x in partition))
        
        if sorted_part not in seen:
            seen.add(sorted_part)
            unique_partitions.append(partition)
    
    memo[alpha_key] = unique_partitions

    return unique_partitions

def memolength(memo):
    """CALCULATES LENGTH OF MEMO"""
    lmemo = {}
    for key in memo:
        lmemo[key] = len(memo[key])
    return lmemo


def Partitions(alpha, embeddings):
    memo={}
    if alpha.is_totally_positive():
        partitions = allParts(alpha,embeddings,memo)
        return len(partitions)+1,memo
    else:
        return 'The number given is not totally positive'




#R.<x> = QQ[]
#K.<x> = NumberField(x^3-3*x-1)
#O = K.ring_of_integers()
#embeddings = K.embeddings(QQbar)  

#alpha=5+x
#alpha1 = O.coordinates(alpha)[0]*O.basis()[0] + O.coordinates(alpha)[1]*O.basis()[1]

#l,fmemo=Partitions(alpha1, embeddings)

#print(fmemo)


In [4]:
import csv

def Partition_Function(Poly, a, b,c):
    R.<x> = QQ[]
    K.<x> = NumberField(Poly)
    O = K.ring_of_integers()
    embeddings = K.embeddings(QQbar) 
    alpha = a + b * x+ c * x^2
    l, fmemo = Partitions(alpha, embeddings)
    
    # Writing to CSV file
    with open("Cubic_x3-3x-1_4.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([b,c,l])

In [5]:
R.<x> = QQ[]
Poly=x^3-3*x-1
for i in range(7):
    for j in range(7):
        try:
            Partition_Function(Poly,4,i,j)
        except ValueError:
            continue



KeyboardInterrupt: 